# How to FineTune Llama 3 with  SFTTrainer and  Unsloth
Hello everyone, today we are going to show how we can Fine Tune Llama 3 with SFTTrainer and Unsloth
First we are going to perform a simmple Fine Tunning by using SFTTrainer


## Step 1 - Installation of Pytorch
The first step is install pythorch v 2.2.1 with Cuda 12.1 

In [ ]:
!python -m pip install --upgrade pip
!pip3 install torch==2.2.1 torchvision torchaudio   xformers --index-url https://download.pytorch.org/whl/cu121

## Step 3 - Installation of Uslotch packages

In [ ]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft \
    accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps  trl peft accelerate bitsandbytes
!pip install datasets
!pip install hyperopt
!pip install optuna    
pass

## Step 4 - Analysis of our infrastructure
In ordering to perform any training it is important to know our system in order to take the full advantage of the system.

In [3]:
!python -m xformers.info
!python -m bitsandbytes
!nvidia-smi

Unable to find python bindings at /usr/local/dcgm/bindings/python3. No data will be captured.
xFormers 0.0.25
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.decoderF:               available
memory_efficient_attention.flshattF@v2.5.6:        available
memory_efficient_attention.flshattB@v2.5.6:        available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.triton_splitKF:         unavailable
indexing.scaled_index_addF:                        unavailable
indexing.scaled_index_addB:                        unavailable
indexing.index_select:

## Step 5 Login to Hugging Face

In [4]:

from huggingface_hub import login, logout
login(token) # non-blocking login

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/wsuser/.cache/huggingface/token
Login successful


**What is SFTTrainer?**

`SFTTrainer` is a class from the `trl` library that implements the SFT algorithm. It is a specialized trainer class that is designed to work with the SFT method. The `SFTTrainer` class takes in a pre-trained model, a dataset, and a set of hyperparameters, and fine-tunes the model using the SFT algorithm.

**What is the difference between SFTTrainer and Trainer?**

The main difference between `SFTTrainer` and the `Trainer` class from the `transformers` library is the fine-tuning algorithm used. The `Trainer` class uses the standard fine-tuning algorithm, where all the model's weights are updated during training. In contrast, the `SFTTrainer` class uses the SFT algorithm, which only updates a small subset of the model's weights. This makes `SFTTrainer` more efficiend suitable for large language models.

**Key differences between SFTTrainer and Trainer**

Here is a table summarizing the key differences between `SFTTrainer` and `Trainer`:

|  | SFTTrainer | Trainer |
| --- | --- | --- |
| Fine-tuning algorithm | Sparse Fine-Tuning (SFT) | Standard fine-tuning |
| Weights updated | Only a small subset of weights | All weights |
| Efficiency | More efficient for large models | Less efficient for large models |
| Suitable for | Large language models | Small to medium-sized models |
| Library | `trl` library | `transformers` library |

Features

| Feature | SFTTrainer | Trainer |
| --- | --- | --- |
| Complexity | Simple, lightweight | More comprehensive, feature-rich |
| Customization | Limited options | Advanced customization options |
| Ease of use | Easy to use, minimal code | More code required, steeper learning curve |
| Integration | Standalone trainer | Part of Hugging Face Transformers library |
| Use cases | Quick fine-tuning, prototyping | Large-scale training, complex models |

Note that the `SFTTrainer` class is specifically designed for sparse fine-tuning, while the `Trainer` class is a more general-purpose trainer class that can be used for a variety of fine-tuning tasks.

##  How to FineTune with SFTTrainer

First let us show the simplest method that is given by  SFTTrainer

In [1]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments
# Load the dataset
dataset_name = "ruslanmv/ai-medical-dataset"
dataset = load_dataset(dataset_name, split="train")
# Select the first 1000 rows of the dataset
dataset = dataset.select(range(100))
# Device map
device_map = 'auto'  # for PP and running with `python test_sft.py`
# Load the model + tokenizer
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache=False,
    device_map=device_map
)
# PEFT config
lora_alpha = 16
lora_dropout = 0.1
lora_r = 32  # 64
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["k_proj", "q_proj", "v_proj", "up_proj", "down_proj", "gate_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm", "norm"],
)
# Args
max_seq_length = 512
output_dir = "./results"
per_device_train_batch_size = 2  # reduced batch size to avoid OOM
gradient_accumulation_steps = 2
optim = "adamw_torch"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1  
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,  # gradient checkpointing
    #report_to="wandb",
)
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="context",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Train :)
trainer.train()


# Save model local
trainer.save_model("ai-medical-model")
tokenizer.save_pretrained("ai-medical-model")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.346700


TrainOutput(global_step=1, training_loss=2.346719980239868, metrics={'train_runtime': 3.3785, 'train_samples_per_second': 1.184, 'train_steps_per_second': 0.296, 'total_flos': 74593973698560.0, 'train_loss': 2.346719980239868, 'epoch': 0.04})

## Inference 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
max_seq_length = 2048
dtype = None
load_in_4bit = True
fine_tuned_model = AutoModelForCausalLM.from_pretrained("ai-medical-model", load_in_4bit=load_in_4bit)
tokenizer = AutoTokenizer.from_pretrained("ai-medical-model")
# Prepare the model for inference
fine_tuned_model.eval()




/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [16]:
question="This is the question: What was the main cause of the inflammatory CD4+ T cells?"
prompt=f"<|start_header_id|>system<|end_header_id|> You are a Medical AI chatbot assistant .<|eot_id|><|start_header_id|> User: <|end_header_id|>{question}<|eot_id|>"
# Tokenizing the input and generating the output
#prompt = f"{question}"
# Tokenizing the input and generating the output
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = fine_tuned_model.generate(**inputs, max_new_tokens=256, use_cache=True)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
answer

'system You are a Medical AI chatbot assistant. User: This is the question: What was the main cause of the inflammatory CD4+ T cells?assistant\n\nThe main cause of the inflammatory CD4+ T cells is often associated with the activation of immune cells, particularly T cells, in response to various stimuli. CD4+ T cells, also known as T helper cells, play a crucial role in the immune system by helping to coordinate the immune response against pathogens, such as bacteria, viruses, and fungi.\n\nThe activation of CD4+ T cells can be triggered by various factors, including:\n\n1. **Infection**: CD4+ T cells recognize and respond to specific antigens, such as those from viruses, bacteria, or fungi, which triggers an inflammatory response.\n2. **Autoimmune disorders**: In autoimmune diseases, such as rheumatoid arthritis, lupus, or multiple sclerosis, CD4+ T cells mistakenly target healthy tissues, leading to chronic inflammation.\n3. **Allergic reactions**: CD4+ T cells play a role in allergic

In [42]:
# Split the text at the first double newline after "assistant\n\n"
split_text = answer.split("\n\n", 2)[1]
# Print the extracted answer
print(split_text.strip())

The main cause of inflammatory CD4+ T cells is a topic of ongoing research and debate in the medical field. However, based on current scientific understanding, it is believed that the main cause of inflammatory CD4+ T cells is the activation of these cells by specific antigens or pathogens, leading to an excessive or uncontrolled immune response.


```txt

The main cause of inflammatory CD4+ T cells is a topic of ongoing research and debate in the medical field. However, based on current scientific understanding, it is believed that the main cause of inflammatory CD4+ T cells is the activation of these cells by specific antigens or pathogens, leading to an excessive or uncontrolled immune response.

```

# How to FineTune with Unsloth
Hello everyone, today we are going to show how we can Fine Tune Llama 3 with a Usloth package.

## Step 5 -  Loading packages
Once we have installed all the packages we load the packages.

In [1]:
import json
import torch
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel
print(torch.__version__)
print(torch.version.cuda)

from huggingface_hub import login, logout
login(token) # non-blocking login

2.2.1+cu121
12.1
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/wsuser/.cache/huggingface/token
Login successful


## Step 6 -  Setup configuration


**Model Configuration**


In [2]:
model_config={ "model_config": {
    "base_model": "meta-llama/Meta-Llama-3-8B-Instruct",  # The base model
    "finetuned_model": "ruslanmv/Medical-Mind-Llama-3-8b-1M",  # The finetuned model
    "finetuned_name": "Medical-Mind-Llama-3-8b-v1M",
    "max_seq_length": 2048,  # The maximum sequence length
    "dtype": None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    "load_in_4bit": True,  # Load the model in 4-bit
}}

* `base_model`: specifies the pre-trained model to use as the base model for fine-tuning.
* `finetuned_model`: specifies the finetuned model to use for fine-tuning.
* `finetuned_name`: specifies the name of the finetuned model.
* `max_seq_length`: specifies the maximum sequence length that the model can process.
* `dtype`: specifies the data type to use for the model's weights and activations. `None` means auto-detection, which will choose the most suitable data type based on the hardware.
* `load_in_4bit`: specifies whether to load the model i 4-bit precision, which can reduce memory usage and improve performance.


**LoRA Configuration**

In [3]:
lora_config={"lora_config": {
    "r": 16,  # The number of LoRA layers 8, 16, 32, 64
    "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],  # The target modules
    "lora_alpha": 16,  # The alpha value for LoRA
    "lora_dropout": 0,  # The dropout value for LoRA
    "bias": "none",  # The bias for LoRA
    "use_gradient_checkpointing": True,  # Use gradient checkpointing
    "use_rslora": False,  # Use RSLora
    "use_dora": False,  # Use DoRa
    "loftq_config": None  # The LoFTQ configuration
}
}

* `r`: specifies the number of LoRA layers to use.
* `target_modules`: specifies the modules to which LoRA should be applied.
* `lora_alpha`: specifies the alpha value for LoRA, which controls the strength of the LoRA layers.
* `lora_dropout`: specifies the dropout value for LoRA, which controls the random dropping of neurons during training.
* `bias`: specifies the bias for LoRA, which can be set to "none" or a specific value.
* `use_gradient_checkpointing`: specifies whether to use gradient checkpointing, which can reduce memory usage during training.
* `use_rslora` and `use_dora`: specify whether to use RSLora and DoRa, respectively, which are variants of LoRA.
* `loftq_config`: specifies the LoFTQ configuration, which is not used in this example.


**Training Configuration**

In [4]:
training_config={"training_config": {
    "per_device_train_batch_size": 2,  # The batch size
    "gradient_accumulation_steps": 4,  # The gradient accumulation steps
    "warmup_steps": 5,  # The warmup steps
    "max_steps": 0,  # The maximum steps (0 if the epochs are defined)
    "num_train_epochs": 1,  # The number of training epochs
    "learning_rate": 2e-4,  # The learning rate
    "fp16": not torch.cuda.is_bf16_supported(),  # The fp16
    "bf16": torch.cuda.is_bf16_supported(),  # The bf16
    "logging_steps":  1,  # The logging steps
    "optim": "adamw_8bit",  # The optimizer
    "weight_decay": 0.0,  # The weight decay
    "lr_scheduler_type": "linear",  # The learning rate scheduler
    "seed": 42,  # The seed
    "output_dir": "outputs",  # The output directory
}
}

* `per_device_train_batch_size`: specifies the batch size to use for training.
* `gradient_accumulation_steps`: specifies the number of steps to accumulate gradients before updating the model.
* `warmup_steps`: specifies the number of warmup steps to perform before starting training.
* `max_steps`: specifies the maximum number of steps to train for. If set to 0, the model will train for the specified number of epochs.
* `num_train_epochs`: specifies the number of epochs to train for.
* `learning_rate`: specifies the initial learning rate to use for training.
* `fp16` and `bf16`: specify whether to use 16-bit floating-point precision (fp16) or 16-bit bfloat precision (bf16) for training.
* `logging_steps`: specifies the number of steps to log training metrics.
* `optim`: specifies the optimizer to use for training.
* `weight_decay`: specifies the weight decay rate to use for regularization.
* `lr_scheduler_type`: specifies the learning rate scheduler to use.
* `seed`: specifies the random seed to use for training.
* `output_dir`: specifies the output directory to save training results.

**Hugging Face Username**

In [5]:
hugging_face_username={"hugging_face_username": "ruslanmv"}


**Training Dataset**

In [6]:
training_dataset={"training_dataset": {
        "name": "ruslanmv/ai-medical-dataset", # The dataset name(huggingface/datasets)
        "split": "train",  # The dataset split
        "input_fields": ["question", "context"] ,# The input fields
        "input_field": "text",# The input field
    },
                }

**`training_dataset`**: This is the top-level key for the dataset configuration.

**`name`**: This specifies the name of the dataset. In this case, it's `ruslanmv/ai-medical-dataset`, which is a dataset hosted on the Hugging Face Hub. The format is `username/dataset_name`.

**`split`**: This specifies the split of the dataset to use for training. In this case, it's set to `"train"`, which means the model will be trained on the training split of the dataset.

**`input_fields`**: This specifies the input fields of the dataset that will be used for trainine, it's a list containing two fields: `"question"` and `"context"`. These fields are likely to be the input features of the dataset.

**`input_field`**: This specifies the primary input field of the dataset. In this case, it's set to `"text"`. This field is likely to be the text input that the model will process.

Here's an example of what this dataset might look like:

| question | context | text |
| --- | --- | --- |
| How does COVID-19 spread? | COVID-19 is a respiratory disease... | The COVID-19 is.. |
| ... | ... | ... |

In [7]:
config = {}
config.update(hugging_face_username)
config.update(model_config)
config.update(lora_config)
config.update(training_config)
config.update(training_dataset)

In [8]:
import json
def save_config_to_json(config, file_path):
    with open(file_path, 'w') as f:
        json.dump(config, f, indent=4)
file_path = "original_config.json"
save_config_to_json(config, file_path)

In [9]:
# Loading the model and the tokenizer for the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config["model_config"].get("base_model"),
    max_seq_length=config["model_config"].get("max_seq_length"),
    dtype=config["model_config"].get("dtype"),
    load_in_4bit=config["model_config"].get("load_in_4bit"),
)

Unsloth: You passed in `meta-llama/Meta-Llama-3-8B-Instruct` and `load_in_4bit = True`.
We shall load `unsloth/llama-3-8b-Instruct-bnb-4bit` for 4x faster loading.
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla V100-PCIE-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Setup for QLoRA/LoRA peft of the base model
model = FastLanguageModel.get_peft_model(
    model,
    r = config.get("lora_config").get("r"),
    target_modules = config.get("lora_config").get("target_modules"),
    lora_alpha = config.get("lora_config").get("lora_alpha"),
    lora_dropout = config.get("lora_config").get("lora_dropout"),
    bias = config.get("lora_config").get("bias"),
    use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
    random_state = 42,
    use_rslora = config.get("lora_config").get("use_rslora"),
    use_dora = config.get("lora_config").get("use_dora"),
    loftq_config = config.get("lora_config").get("loftq_config"),
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [12]:
#from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained(config.get("model_config").get("base_model"))
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [13]:
is_test=True
import datasets
import os
dataset_path = "train_dataset"
if os.path.exists(dataset_path):
    print("Dataset exists!")
    train_dataset = datasets.load_from_disk("train_dataset")
else:
    print("Dataset does not exist.")
    # Loading the training dataset
    train_dataset = load_dataset(config.get("training_dataset").get("name"), split = config.get("training_dataset").get("split"))    
    
    if is_test:
        # Select the first 1M rows of the dataset
        train_dataset = train_dataset.select(range(100))
        
    medical_prompt = """You are an AI Medical Assistant Chatbot, trained to answer medical questions. Below is an instruction that describes a task, paired with an response context. Write a response that appropriately completes the request.
    ### Instruction:
    {}

    ### Response:
    {}"""
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
    def formatting_prompts_func(examples):
        instructions = examples["question"]
        outputs      = examples["context"]
        texts = []
        for instruction, output in zip(instructions,  outputs):
            # Must add EOS_TOKEN, otherwise your generation will go on forever!
            text = medical_prompt.format(instruction,  output) + EOS_TOKEN
            texts.append(text)
        return { "text" : texts, }
    pass
    train_dataset= train_dataset.map(formatting_prompts_func, batched = True,)
    train_dataset['text'][1]    
    import datasets
    # assume 'test_dataset' is a Dataset object
    train_dataset.save_to_disk("train_dataset")    

Dataset does not exist.


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
train_dataset

Dataset({
    features: ['question', 'context', 'text'],
    num_rows: 100
})

In [15]:
is_multiple=True

In [34]:
if is_multiple:
    # Set up GPU acceleration
    if torch.cuda.device_count() > 1:
        print("Multiple GPUs enabled")
        devices = [f"cuda:{i}" for i in range(torch.cuda.device_count())]
        model_parallel = torch.nn.DataParallel(model, device_ids= devices ) #[0, 1]
        # Access the original model from the DataParallel object
        model = model_parallel.module
    else:
        print("No DataParallel ")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)        

Multiple GPUs enabled


In [33]:
# Setting up the trainer for the model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = config.get("training_dataset").get("input_field"),
    max_seq_length = config.get("model_config").get("max_seq_length"),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = config.get("training_config").get("per_device_train_batch_size"),
        gradient_accumulation_steps = config.get("training_config").get("gradient_accumulation_steps"),
        warmup_steps = config.get("training_config").get("warmup_steps"),
        max_steps = config.get("training_config").get("max_steps"),
        num_train_epochs= config.get("training_config").get("num_train_epochs"),
        learning_rate = config.get("training_config").get("learning_rate"),
        fp16 = config.get("training_config").get("fp16"),
        bf16 = config.get("training_config").get("bf16"),
        logging_steps = config.get("training_config").get("logging_steps"),
        optim = config.get("training_config").get("optim"),
        weight_decay = config.get("training_config").get("weight_decay"),
        lr_scheduler_type = config.get("training_config").get("lr_scheduler_type"),
        seed = 42,
        output_dir = config.get("training_config").get("output_dir"),
        
    ),
)

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [26]:
# Memory statistics before training
gpu_statistics = torch.cuda.get_device_properties(0)
reserved_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
max_memory = round(gpu_statistics.total_memory / 1024**3, 2)
print(f"Reserved Memory: {reserved_memory}GB")
print(f"Max Memory: {max_memory}GB")

Reserved Memory: 11.19GB
Max Memory: 15.77GB


In [27]:
##  [ 1038/2651250 53:49 < 2295:10:28, 0.32 it/s, Epoch 0.00/1] old

In [ ]:
# Training the model
trainer_stats = trainer.train()

In [ ]:
# Memory statistics after training
used_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
used_memory_lora = round(used_memory - reserved_memory, 2)
used_memory_persentage = round((used_memory / max_memory) * 100, 2)
used_memory_lora_persentage = round((used_memory_lora / max_memory) * 100, 2)
print(f"Used Memory: {used_memory}GB ({used_memory_persentage}%)")
print(f"Used Memory for training(fine-tuning) LoRA: {used_memory_lora}GB ({used_memory_lora_persentage}%)")

In [ ]:
new_model=config.get("model_config").get("finetuned_model")

In [ ]:
new_model

In [ ]:
# Saving the trainer stats
with open("trainer_stats.json", "w") as f:
    json.dump(trainer_stats, f, indent=4)

In [ ]:
## Save and push the adapter to HF
import os
current_directory = os.getcwd()
# New model name
new_model = config.get("model_config").get("finetuned_name") #"Medical-Mind-Llama-3-8b"
# Save the fine-tuned model
save_path = os.path.join(current_directory, "models", new_model)

In [ ]:
#os.makedirs(save_path, exist_ok=True)  # Create directory if it doesn't exist
#trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
help(model.save_pretrained_merged)

To save the final model as LoRA adapters, either use Huggingface's push_to_hub for an online save or save_pretrained for a local save.

[NOTE] This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# Save the model to the created directory
# `lora`: Save LoRA adapters with no merging. Useful for HF inference.
#model.save_pretrained(save_path)


In [28]:
# Saving the model using merged_16bit(float16), 
#`16bit`: Merge LoRA into float16 weights. Useful for GGUF / llama.cpp.
#model.save_pretrained_merged(save_path, tokenizer, save_method = "merged_16bit")

In [ ]:
# `4bit`: Merge LoRA into int4 weights. Useful for DPO / HF inference.
model.save_pretrained_merged(save_path, tokenizer, save_method = "merged_4bit_forced")

In [ ]:
save_path

In [ ]:
# Get the list of files in the directory
files_in_model_dir = os.listdir(save_path)
# Print the list of files
print("Files in the directory:")
for file in files_in_model_dir:
    print(file)

In [ ]:
import os
from huggingface_hub import HfApi
def upload_folder(folder_path, repository_name, path_in_repo):
    api = HfApi()
    
    # Check if the repository exists, if not, create it
    repo_exists = api.repo_exists(repository_name)
    if not repo_exists:
        api.create_repo(repository_name)
        print(f"Repository '{repository_name}' created on Hugging Face Hub.")

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, folder_path)
            repo_path = os.path.join(path_in_repo, relative_path)
            api.upload_file(path_or_fileobj=file_path, repo_id=repository_name, path_in_repo=repo_path)
            print(f"{repo_path} uploaded to {repository_name}")

In [ ]:
# Define the repository name and path in the repository
repository_name = "ruslanmv/"+new_model
path_in_repo = ""

In [ ]:
repository_name

In [ ]:
# Upload the folder and its contents to the repository
upload_folder(save_path, repository_name, path_in_repo)

In [ ]:
#help(model.push_to_hub_merged)

In [ ]:
#save_path='/home/wsuser/work/models/Medical-Mind-Llama-3-8b'
#repo_id='ruslanmv/Medical-Mind-Llama-3-8b'
#commit_message="Uploading Model"
#model.push_to_hub_merged(repo_id, tokenizer, save_method = "merged_16bit",commit_message=commit_message)

In [ ]:
#model.push_to_hub_merged(config.get("model_config").get("finetuned_model"), tokenizer, save_method = "merged_4bit")

In [ ]:
#model.save_pretrained_gguf(config.get("model_config").get("finetuned_model"), tokenizer)
#model.push_to_hub_gguf(config.get("model_config").get("finetuned_model"), tokenizer,repository_private=True)

In [ ]:
#model.save_pretrained_gguf(config.get("model_config").get("finetuned_model"), tokenizer, quantization_method = "q4_k_m")
#model.push_to_hub_gguf(config.get("model_config").get("finetuned_model"), tokenizer, quantization_method = "q4_k_m",private=True)

In [5]:
is_inference=False
if is_inference:
    from unsloth import FastLanguageModel
    import torch
    max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
    dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "ruslanmv/Medical-Mind-Llama-3-8b-1M",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    # Using FastLanguageModel for fast inference
    FastLanguageModel.for_inference(model)
    question="This is the question: What was the main cause of the inflammatory CD4+ T cells?"
    prompt=f"<|start_header_id|>system<|end_header_id|> You are a Medical AI chatbot assistant .<|eot_id|><|start_header_id|> user <|end_header_id|>{question}<|eot_id|>"
    # Tokenizing the input and generating the output
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  # Get the first element from the batch

    # Split the answer at the first line break, assuming system intro and question are on separate lines
    answer_parts = answer.split("\n", 1)

    # If there are multiple parts, consider the second part as the answer
    if len(answer_parts) > 1:
      answer = answer_parts[1].strip()  # Remove leading/trailing whitespaces
    else:
      answer = ""  # If no split possible, set answer to empty string

    print(f"Answer: {answer}")    
 

/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla V100-PCIE-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: I'd be happy to help you with that!

The main cause of inflammatory CD4+ T cells is a complex process involving multiple factors. CD4+ T cells, also known as T helper cells, play a crucial role in the immune response. Inflammation occurs when these cells become activated and produce pro-inflammatory cytokines, leading to an imbalance in the immune response.

Some common causes of inflammatory CD4+ T cells include:

1. **Infections**: Bacterial, viral, or fungal infections can trigger an immune response, leading to the activation of CD4+ T cells and subsequent inflammation.
2. **Autoimmune disorders**: Conditions like rheumatoid arthritis, lupus, or multiple sclerosis can cause CD4+ T cells to become chronically activated, leading to chronic inflammation.
3. **Allergies**: Allergic reactions can trigger the activation of CD4+ T cells, resulting in the release of pro-inflammatory cytokines and the development of inflammation.
4. **Cancer**: Tumor cells can stimulate the activatio

# Hyperparameter search
**Step 1: Define the Hyperparameter Search Space**
We need to define the search space for the hyperparameters we want to tune. For example, let's say we want to tune the following hyperparameters:

* `learning_rate`
* `per_device_train_batch_size`
* `gradient_accumulation_steps`
* `warmup_steps`
* `num_train_epochs`
* `lora_alpha`
* `lora_dropout`

We can define the search space as follows:

In [ ]:
import hyperopt
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin, tpe, Trials
# Define the search space for hyperparameters
space = {
  'learning_rate': hp.loguniform('learning_rate', -5, -1),  # Learning rate in log scale
  #'lora_alpha': hp.quniform('lora_alpha', 1, 32, 1),  # LoRA alpha with quantized steps
  #'lora_dropout': hp.uniform('lora_dropout', 0, 0.5),  # LoRA dropout rate

  'per_device_train_batch_size': hp.quniform('per_device_train_batch_size', 2, 16, q=1),  
  'gradient_accumulation_steps': hp.quniform('gradient_accumulation_steps', 1, 8, 1),  # Added for exploration
  # Uncomment these if you want to tune other hyperparameters
  # 'warmup_steps': hp.quniform('warmup_steps', 0, 1000, 1),
  # 'num_train_epochs': hp.quniform('num_train_epochs', 1, 5, 1),    

}

**Step 2. Define the Objective Function**

The objective function is a function that takes in the hyperparameters, sets them in the `config` dictionary, trains the model, and returns the loss or metric to minimize. We need to modify the previous fine-tuning code to define the objective function.

In [ ]:
def objective(params):
  # Set hyperparameters in the config dictionary (assuming it's defined elsewhere)
  config['training_config']['learning_rate'] = params['learning_rate']
 # config['lora_config']['lora_alpha'] = params['lora_alpha']
 # config['lora_config']['lora_dropout'] = params['lora_dropout']   
  config['training_config']['per_device_train_batch_size'] = params['per_device_train_batch_size']
  config['training_config']['gradient_accumulation_steps'] = params['gradient_accumulation_steps']
  # ... Set other hyperparameters from params dictionary ...   
  #config['training_config']['warmup_steps'] = params['warmup_steps']
  #config['training_config']['num_train_epochs'] = params['num_train_epochs']

  # Load the model and tokenizer (assuming these are defined elsewhere)
  try:
      model, tokenizer = FastLanguageModel.from_pretrained(
          model_name=config.get("model_config").get("base_model"),
          max_seq_length=config.get("model_config").get("max_seq_length"),
          dtype=config.get("model_config").get("dtype"),
          load_in_4bit=config.get("model_config").get("load_in_4bit"),
      )
  except Exception as e:
      print(f"Error loading model and tokenizer: {e}")
      return float("inf")  # Return high value for errors

  # Setup LoRA for the model (assuming FastLanguageModel supports LoRA)
  try:
      model = FastLanguageModel.get_peft_model(
          model,
          r=config.get("lora_config").get("r"),
          target_modules=config.get("lora_config").get("target_modules"),
          lora_alpha=params['lora_alpha'],
          lora_dropout=params['lora_dropout'],
          bias=config.get("lora_config").get("bias"),
          use_gradient_checkpointing=config.get("lora_config").get("use_gradient_checkpointing"),
          random_state=42,
          use_rslora=config.get("lora_config").get("use_rslora"),
          use_dora=config.get("lora_config").get("use_dora"),
          loftq_config=config.get("lora_config").get("loftq_config")
      )
  except Exception as e:
      print(f"Error setting up LoRA: {e}")
      return float("inf")  # Return high value for errors
  # Train the model on the test dataset (assuming SFTTrainer and training arguments are defined)
  try:
      trainer = SFTTrainer(
          model=model,
          tokenizer=tokenizer,
          train_dataset=train_dataset,
          dataset_text_field=config.get("training_dataset").get("input_field"),
          max_seq_length=config.get("model_config").get("max_seq_length"),
          dataset_num_proc=2,
          packing=False,
          args=TrainingArguments(
              per_device_train_batch_size=int(params['per_device_train_batch_size']),
              gradient_accumulation_steps=params['gradient_accumulation_steps'],
              warmup_steps=params['warmup_steps'],
              max_steps=config.get("training_config").get("max_steps"),
              num_train_epochs=params['num_train_epochs'],
              learning_rate=params['learning_rate'],
              fp16=config.get("training_config").get("fp16"),
              bf16=config.get("training_config").get("bf16"),
              logging_steps=config.get("training_config").get("logging_steps"),
              optim=config.get("training_config").get("optim"),
              weight_decay=config.get("training_config").get("weight_decay"),
              lr_scheduler_type=config.get("training_config").get("lr_scheduler_type"),
              seed=42,
              output_dir=config.get("training_config").get("output_dir")
          )
      )
      trainer_stats = trainer.train()
      return trainer_stats.loss  # Assuming loss is the metric to minimize
  except Exception as e:
      print(f"Error during training: {e}")
      return float("inf")  # Return high value for failed trials



**Step 3: Perform Hyperparameter Search**

Now that we have defined the objective function, we can perform the hyperparameter search using Hyperopt's `fmin` function. We need to specify the objective function, the search space, and the maximum number of evaluations.

In [ ]:

# Create a Trials object to track hyperparameter evaluations
trials = Trials()
# Run hyperparameter optimization using TPE algorithm
best = fmin(objective, space, algo=tpe.suggest, trials=trials, max_evals=2)
# Print the best hyperparameters found during optimization
print("Best Hyperparameters:", best)

In [ ]:
import torch
import gc
def reset_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()
    print("GPU memory cleared!")
# Example usage:
reset_gpu_memory()

## Full code version

In [ ]:
####  Fixed Version
import hyperopt
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin, tpe, Trials
# Define the search space for hyperparameters
space = {
  'learning_rate'              : hp.loguniform('learning_rate', -5, -1),  # Learning rate in log scale
  'per_device_train_batch_size': hp.quniform('per_device_train_batch_size', 2, 16, 1),   
  'gradient_accumulation_steps': hp.quniform('gradient_accumulation_steps', 1, 8, 1), 
  # Uncomment these if you want to tune them
  #'lora_alpha'                : hp.quniform('lora_alpha', 1, 32, 1),  # LoRA alpha with quantized steps
  #'lora_dropout'              : hp.uniform('lora_dropout', 0, 0.5),  # LoRA dropout rate
  # 'warmup_steps'             : hp.quniform('warmup_steps', 0, 1000, 1),
  # 'num_train_epochs'         : hp.quniform('num_train_epochs', 1, 5, 1),
}
def objective(params):
    # Set hyperparameters in the config dictionary (assuming it's defined elsewhere)
    config['training_config']['learning_rate']=params['learning_rate']
    config['training_config']['per_device_train_batch_size'] = params['per_device_train_batch_size']
    config['training_config']['gradient_accumulation_steps'] = params['gradient_accumulation_steps']    
    # config['lora_config']['lora_alpha'] = params['lora_alpha']
    # config['lora_config']['lora_dropout'] = params['lora_dropout']
    # ... Set other hyperparameters from params dictionary ...
    #config['training_config']['warmup_steps'] = params['warmup_steps']
    #config['training_config']['num_train_epochs'] = params['num_train_epochs']
    # Load the model and tokenizer (assuming these are defined elsewhere)    
    try:
            model, tokenizer    = FastLanguageModel.from_pretrained(
            model_name          = config.get("model_config").get("base_model"),
            max_seq_length      = config.get("model_config").get("max_seq_length"),
            dtype               = config.get("model_config").get("dtype"),
            load_in_4bit        = config.get("model_config").get("load_in_4bit")
            )
    except Exception as e:
        print(f"Error loading model and tokenizer: {e}")
        return float("inf")  # Return high value for errors

    # Setup LoRA for the model (assuming FastLanguageModel supports LoRA)
    try:
        model = FastLanguageModel.get_peft_model(
        model,
        r                            = config.get("lora_config").get("r"),
        target_modules               = config.get("lora_config").get("target_modules"),
        lora_alpha                   = config.get("lora_config").get('lora_alpha'), #params['lora_alpha'],
        lora_dropout                 = config.get("lora_config").get('lora_dropout'),#params['lora_dropout'],
        bias                         = config.get("lora_config").get("bias"),
        use_gradient_checkpointing   = config.get("lora_config").get("use_gradient_checkpointing"),
        random_state                 = 42,
        use_rslora                   = config.get("lora_config").get("use_rslora"),
        use_dora                     = config.get("lora_config").get("use_dora"),
        loftq_config                 = config.get("lora_config").get("loftq_config")
        )
    except Exception as e:
        print(f"Error setting up LoRA: {e}")
        return float("inf")  # Return high value for errors
    # Train the model on the test dataset (assuming SFTTrainer and training arguments are defined)
    try:
        trainer = SFTTrainer(
              model=model,
              tokenizer            =  tokenizer,
              train_dataset        = train_dataset,
              dataset_text_field   = config.get("training_dataset").get("input_field"),
              max_seq_length       = config.get("model_config").get("max_seq_length"),
              dataset_num_proc     = 2,
              packing              = False,
              args=TrainingArguments(
                  per_device_train_batch_size = int(params['per_device_train_batch_size']), #config.get("training_config").get('per_device_train_batch_size'),
                  gradient_accumulation_steps = params['gradient_accumulation_steps'], #config.get("training_config").get('gradient_accumulation_steps'),
                  warmup_steps                = config.get("training_config").get('warmup_steps'),#params['warmup_steps'],
                  max_steps                   = config.get("training_config").get("max_steps"),
                  num_train_epochs            = config.get("training_config").get('num_train_epochs'),#params['num_train_epochs'],
                  learning_rate               = params['learning_rate'],
                  fp16                        = config.get("training_config").get("fp16"),
                  bf16                        = config.get("training_config").get("bf16"),
                  logging_steps               = config.get("training_config").get("logging_steps"),
                  optim                       = config.get("training_config").get("optim"),
                  weight_decay                = config.get("training_config").get("weight_decay"),
                  lr_scheduler_type           = config.get("training_config").get("lr_scheduler_type"),
                  seed                        = 42,
                  output_dir                  = config.get("training_config").get("output_dir")
              )
          )
        trainer_stats = trainer.train()
        return trainer_stats.loss  # Assuming loss is the metric to minimize
    except Exception as e:
        print(f"Error during training: {e}")
        return float("inf")  # Return high value for failed trials    
# Create a Trials object to track hyperparameter evaluations
trials = Trials()
# Run hyperparameter optimization using TPE algorithm
best = fmin(objective, space, algo=tpe.suggest, trials=trials, max_evals=2)
# Print the best hyperparameters found during optimization
print("Best Hyperparameters:", best)   

## Analyzing Hyperparameters:

*  **Batch Size**: Generally, increasing the batch size can improve training speed by utilizing hardware resources more efficiently. However, there's a limit beyond which performance degrades. You can tune the batch size within a reasonable range (e.g., 2, 4, 8, 16) to see its impact.

* **Learning Rate**: A higher learning rate can accelerate training initially. But, a too high value can lead to unstable training and potentially slower convergence. Consider a range of learning rates (e.g., log-uniform distribution between 1e-5 and 1e-3) for exploration.

* **Gradient Accumulation Steps**: This technique accumulates gradients over multiple batches before updating model weights. It can help reduce memory requirements but might slow down training per epoch. Experiment with different accumulation steps (e.g., 1, 2, 4) to find a balance.

* **Optimizer Choice**: Some optimizers like Adam or SGD with momentum can be faster than others depending on the model and dataset. Explore different optimizers and their hyperparameters (e.g., momentum coefficient) to see if they lead to faster convergence.

## Additional Considerations:

Early Stopping: Implement early stopping to automatically terminate training if the validation loss doesn't improve for a certain number of epochs. This can save training time if the model starts overfitting.
Warmup Steps: A gradual increase in the learning rate during the initial training phase (warmup steps) can improve stability and potentially accelerate convergence compared to a fixed learning rate from the beginning.


* Experimentation and Profiling:

The best hyperparameters for faster training depend on your specific model, dataset, and hardware. You'll need to experiment with different configurations using tools like Hyperopt to find the optimal settings.
Consider using profiling tools to identify bottlenecks in your training pipeline. This can help you focus on optimizing specific parts of the training process that are most time-consuming.
By analyzing these hyperparameters and implementing techniques like early stopping and warmup steps, you can potentially achieve faster fine-tuning while maintaining good model performance.

## Method 1  Optuna

In [ ]:
from optuna import create_study, Trial
import time  # Assuming you can use time.time() to measure training time

# Define search space with additional hyperparameter
search_space = {
  "learning_rate": [1e-5, 5e-5, 1e-4, 2e-4],
  "per_device_train_batch_size": [2, 4, 8],
  "lora_alpha": [8, 16, 32],
  "gradient_accumulation_steps": [1, 2, 4, 8],  # Added gradient accumulation steps
}

def objective(trial):
  # Set hyperparameters based on trial values
  config["training_config"]["learning_rate"] = trial.suggest_float("learning_rate", search_space["learning_rate"][0], search_space["learning_rate"][-1])
  config["training_config"]["per_device_train_batch_size"] = trial.suggest_int("per_device_train_batch_size", search_space["per_device_train_batch_size"][0], search_space["per_device_train_batch_size"][-1])
  config["training_config"]["gradient_accumulation_steps"] = trial.suggest_int("gradient_accumulation_steps", search_space["gradient_accumulation_steps"][0], search_space["gradient_accumulation_steps"][-1])
  config["lora_config"]["lora_alpha"] = trial.suggest_int("lora_alpha", search_space["lora_alpha"][0], search_space["lora_alpha"][-1])

  # Train the model with the current hyperparameters
  start_time = time.time()
  try:
      trainer_stats = trainer_test.train()
      training_time = time.time() - start_time
      return training_time  # Minimize training time
  except Exception as e:
      return float("inf")  # Assign a high value if training fails

study = create_study(direction="minimize")
study.optimize(objective, n_trials=2)  # Adjust the number of trials

# Access the best trial and its hyperparameters after optimization
best_trial = study.best_trial
best_params = best_trial.params

print("Best Trial:", best_trial.number)
print("Best Hyperparameters (Likely Fastest):", best_params)
print("Best Training Time:", best_trial.value, "seconds")